In [1]:
# import libraries
from pyopenms import * # main package used for handling MS data
import os # changing directories
import pandas as pd # to read in tsv and for dataframe creation/manipulation
import subprocess # running R script 

In [2]:
# after running main script
# change experiment files in DeNovo_main.py as well for now
from DeNovo_main import *

Experiment Files: 
HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD30.mzML
App-2022-06-12_22-28-53.log
HEK293T_De_Novo_061122_Glu-C_B_BP_anyLength_HCD30_realtimesearch.tsv
-------------------------------------------------
Number of MS2 scans: 81780
Number of MS3 scans: 681
-------------------------------------------------
Precursor mzs match up after taking rounding discrepencies into consideration
Fragment mzs match up perfectly!


In [3]:
new_fragment_df

,Scan_Number,Sequence,Charge,Sequence_Length,Missing_Fragment_Locations,Number_Missing,Target_Fragment,MS3_Scan,Fragment_Sequence,Locations_Found
13,4039,VSLTQKTDPSVRPMHE,4,16,,0,y4,4048,PMHE,[2]
40,4610,AGLNVTTSHSPAAPGE,2,16,"1,3,8,15",4,y3,4621,PGE,[2]
42,4630,AGLNVTTSHSPAAPGE,2,16,1,1,y3,4639,PGE,[2]
55,4986,HASIQMNVAE,2,10,,0,b5,4994,HASIQ,[2]
60,5057,AAKLQTTKVKKPTGTRNLYLARE,5,23,,0,b4,5062,AAKL,[3]
88,5600,FKPNKPKPCGLCNQFGHE,4,18,,0,y3,5602,GHE,[2]
91,5713,GSSIKKAQQAVANKALTE,3,18,,0,b2,5723,GS,[1]
133,7286,IVGGATRIPAVKE,3,13,2,1,y2,7299,KE,[1]
141,7640,KPAKAITSSRVPGE,3,14,,0,b2,7644,KP,[1]
157,8134,RRQLIVPPHLAHGE,4,14,,0,b6,8141,RRQLIV,"[4, 5]"


In [7]:
s = specM3[212]

In [8]:
# obtain mz and intensity values 
mz, intensity = s.get_peaks()

In [27]:
mz_new = []
count = -1
for m in mz:
    count = count + 1
    if m < 293 and m > 149:
        mz_new.append(m)
    else:
        print(count) # indicies of intensities to remove

0
1
2
3
4
5
6
7


In [21]:
mz_new

[149.7283477783203,
 149.74888610839844,
 149.75672912597656,
 157.09690856933594,
 181.6333770751953,
 185.0922393798828,
 292.1139831542969]

In [29]:
intensity_new = np.delete(intensity, [0,1,2,3,4,5,6,7])

In [32]:
base_normalized = [x/intensity_new.max() for x in intensity_new]

In [33]:
base_normalized

[0.2831351, 0.50868934, 0.3603572, 1.0, 0.26224923, 0.9032462, 0.25380728]

In [43]:
# special case when there is an internal y ion in a b fragment
def findFragmentsMS3(peptide_object, charge, i):
    
    y_index = []
    b_index = []

    # targeted fragment is a y ion
    if new_fragment_df['Target_Fragment'][i].startswith('y'):
        y_num = new_fragment_df['Target_Fragment'][i][-1]

        # the full sequence of the fragment
        full_seq = peptide_object.getSuffix(int(y_num))

        # checking fragment for its y ions
        for ion in range(1, int(y_num)):
            y_ion = full_seq.getSuffix(ion)
            for z in range(1, charge):
                mz_y = y_ion.getMonoWeight(Residue.ResidueType.YIon, z) / z
                print(mz_y, 'yy')
                y_index.append(specM3[i].findNearest(mz_y, 0.02))

        # reverse list (the first b ion corresponds with the last y ion)
        y_index.reverse() 

        # checking fragment for b ions
        for ion in range(1, int(y_num)):
            b_ion = full_seq.getPrefix(ion)
            for z in range(1, charge):
                mz_b = b_ion.getMonoWeight(Residue.ResidueType.Internal, z) / z
                print(mz_b, 'yb')
                b_index.append(specM3[i].findNearest(mz_b, 0.02))
                
    # targeted fragment is a b ion
    elif new_fragment_df['Target_Fragment'][i].startswith('b'):
        b_num = new_fragment_df['Target_Fragment'][i][-1]

        # the full sequence of the fragment
        full_seq = peptide_object.getPrefix(int(b_num))

        # checking fragment for its b ions
        for ion in range(1, int(b_num)):
            b_ion = full_seq.getPrefix(ion)
            for z in range(1, charge):
                mz_b = b_ion.getMonoWeight(Residue.ResidueType.BIon, z) / z
                print(mz_b, 'bb')
                b_index.append(specM3[i].findNearest(mz_b, 0.02))

        # checking fragment for y ions
        for ion in range(1, int(b_num)):
            y_ion = full_seq.getSuffix(ion)
            for z in range(1, charge):
                mz_y = y_ion.getMonoWeight(Residue.ResidueType.Internal, z) / z
                print(mz_y, 'by')
                y_index.append(specM3[i].findNearest(mz_y, 0.02))

        # reverse list (the first b ion corresponds with the last y ion)
        y_index.reverse()

    count = 0
    foundinMS3 = []
    for fragment_index in range(0, len(y_index), charge-1): # check after 'x' mzs (possible charge states)

        count = count + 1

        check_y = y_index[fragment_index : fragment_index+charge-1]
        check_b = b_index[fragment_index : fragment_index+charge-1]

        if all(item == -1 for item in check_b) and all(item == -1 for item in check_y): # missing fragments
            pass 
        else:
            foundinMS3.append(count) # fragments found (distance from n terminus of target frag)

    #return mz_b, mz_y

In [44]:
# create peptide object for the sequence
peptide_object = AASequence.fromString(new_fragment_df['Sequence'][212])
    
# the charge associated with this sequence
charge = 2
    
# call findFragmentsMS3 function 
findFragmentsMS3(peptide_object, charge, 212)
    

148.060435753871 yy
205.08189984957102 yy
292.113929009071 yy
98.06004069007099 yb
185.09206984957098 yb
242.11353394527097 yb


In [ ]:
# targeted y ion so special case does not apply 